In [ ]:
#Surreal Adversarial Learning, used here as simply an altered training paradigm that slowly increases a self-sabotoging
#attack on the reward function, primarily here experimenting with how this type of targeted attack can increase both 
#adversarial robustness as well as overall performance via implicit self-supervised learning. 

#Uses a baseline DQN experiment from an author I cannot find, (apologies to the original author...)

#Video series on Surreal Adversarial Learning: https://www.youtube.com/watch?v=gpIJ41IZj_o&list=PLKgB8_r-NpR5YcI1knBLHh7XZeh3ESKvn

# Deep Q-Network

In 2015, Google DeepMind ([Link](https://deepmind.com/research/dqn/)) published a paper in Nature magazine that combines a deep convolution neural network with reinforcement learning for the first time in order to master a range of Atari 2600 games. They used only the raw pixels and score as the inputs. They were able to use the convolution layer to translate the pixels.  

The very simple description is that they replaced the Q table in a Q-Learner with a neural network. This allowed them to take advantage of neural networks but still use reinforcement learning. 

In [ ]:
#Imports
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.optimizers import Adam
import random

#Create Gym
from gym import wrappers
envCartPole = gym.make('CartPole-v1')
envCartPole.seed(50) #Set the seed to keep the environment consistent across runs

[50]

**Experience Replay**  
Definition: A mechanism inspired by biology that randomizes over the data removing the correlation in the observation sequence and smoothing over changes in the data distribution.  

To perform an experience replay, the algorithm stores all of the agents experiences {$s_t,a_t,r_t,s_{t+1}$} at each time step in a data set. Normally in a q-learner, we would run the update rule on them. But, with experience replay we just store them.  

Later during the training process these replays will be drawn uniformly from the memory queue and be ran through the update rule. There are 2 ways to handle this and I have coded both in the past. The first is to run them on every loop and the other is to run them after X amount of runs. In this code below, I run them each time.

**CartPole Example**  
Again we will use the [CartPole](https://gym.openai.com/envs/CartPole-v1/) environment from OpenAI.  

The actions are 0 to push the cart to the left and 1 to push the cart to the right.  

The continuous state space is an X coordinate for location, the velocity of the cart, the angle of the pole, and the velocity at the tip of the pole. The X coordinate goes from -4.8 to +4.8, velocity is -Inf to +Inf, angle of the pole goes from -24 degrees to +24 degrees, tip velocity is -Inf to +Inf. With all of the possible combinations you can see why we can't create a Q table for each one.  

To "solve" this puzzle you have to have an average reward of > 195 over 100 consecutive episodes. One thing to note, I am hard capping the rewards at 210 so this number can't average above that and it also could potentially drive the average down.

In [ ]:
#Global Variables
EPISODES = 500
TRAIN_END = 0

In [ ]:
#Hyper Parameters
#AKA your fiddly bits :)
def discount_rate(): #Gamma
    return 0.95

def learning_rate(): #Alpha
    return 0.001

def batch_size(): #Size of the batch used in the experience replay
    return 24

## **Deep Q-Network Class**  
The following class is the deep Q-network that is built using the neural network code from Keras.  
**init**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This creates the class and sets the local parameters.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;I use a *deque* for the local memory to hold the experiences and a keras model for the NN.  

**build_model(self)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This builds the NN. I am using sequential model. Each of the layers are *Dense* despite the fact the document talks about using *Convolution*. But, they are only using that because they need to convert pixels and I already have numbers.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;I am using an input layer(4), 24 neuron layer, 24 neuron layer, and an output layer(2).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For calculating the loss I am using mean squared error.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For an optimizer I am using [Adam](https://arxiv.org/abs/1412.6980v8). It is a variant of gradient descent and you can read the technical document at the link. If you want a slightly lighter explaining you can check out [Machine Learning Mastery](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/). You could also use SGD (Stochastic Gradient Descent) but Adam gives me better results and seems to be the standard in most examples.  

**action(self,state)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This generates the action.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Explore: I am using the epsilon like previous lessons.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Exploit: I use the NN to grab the 2 possible actions and then grab the argmax to find the better one  

**test_action(self,state)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This generates the action when I am testing. I want to 100% exploit  

**store(self, state, action, reward, nstate, done)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This places the observables in memory  

**experience_replay(self, batch_size)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This is where the training occurs. We grab the sample batches and then use the NN to predict the optimal action.  

In [ ]:
class DeepQNetwork():
    def __init__(self, states, actions, alpha, gamma, epsilon,epsilon_min, epsilon_decay):
        self.nS = states
        self.nA = actions
        self.memory = deque([], maxlen=2500)
        self.alpha = alpha
        self.gamma = gamma
        #Explore/Exploit
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.model = self.build_model()
        self.loss = []
        
    def build_model(self):
        model = keras.Sequential() #linear stack of layers https://keras.io/models/sequential/
        model.add(keras.layers.Dense(24, input_dim=self.nS, activation='relu')) #[Input] -> Layer 1
        #   Dense: Densely connected layer https://keras.io/layers/core/
        #   24: Number of neurons
        #   input_dim: Number of input variables
        #   activation: Rectified Linear Unit (relu) ranges >= 0
        model.add(keras.layers.Dense(24, activation='relu')) #Layer 2 -> 3
        model.add(keras.layers.Dense(self.nA, activation='linear')) #Layer 3 -> [output]
        #   Size has to match the output (different actions)
        #   Linear activation on the last layer
        model.compile(loss='mean_squared_error', #Loss function: Mean Squared Error
                      optimizer=keras.optimizers.Adam(lr=self.alpha)) #Optimaizer: Adam (Feel free to check other options)
        return model

    def action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.nA) #Explore
        action_vals = self.model.predict(state) #Exploit: Use the NN to predict the correct action from this state
        return np.argmax(action_vals[0])

    def test_action(self, state): #Exploit
        action_vals = self.model.predict(state)
        return np.argmax(action_vals[0])

    def store(self, state, action, reward, nstate, done):
        #Store the experience in memory
        self.memory.append( (state, action, reward, nstate, done) )

    def experience_replay(self, batch_size):
        #Execute the experience replay
        minibatch = random.sample( self.memory, batch_size ) #Randomly sample from memory

        #Convert to numpy for speed by vectorization
        x = []
        y = []
        np_array = np.array(minibatch)
        st = np.zeros((0,self.nS)) #States
        nst = np.zeros( (0,self.nS) )#Next States
        for i in range(len(np_array)): #Creating the state and next state np arrays
            st = np.append( st, np_array[i,0], axis=0)
            nst = np.append( nst, np_array[i,3], axis=0)
        st_predict = self.model.predict(st) #Here is the speedup! I can predict on the ENTIRE batch
        nst_predict = self.model.predict(nst)
        index = 0
        for state, action, reward, nstate, done in minibatch:
            x.append(state)
            #Predict from state
            nst_action_predict_model = nst_predict[index]
            if done == True: #Terminal: Just assign reward much like {* (not done) - QB[state][action]}
                target = reward
            else:   #Non terminal
                target = reward + self.gamma * np.amax(nst_action_predict_model)
            target_f = st_predict[index]
            target_f[action] = target
            y.append(target_f)
            index += 1
        #Reshape for Keras Fit
        x_reshape = np.array(x).reshape(batch_size,self.nS)
        y_reshape = np.array(y)
        epoch_count = 1 #Epochs is the number or iterations
        hist = self.model.fit(x_reshape, y_reshape, epochs=epoch_count, verbose=0)
        #Graph Lossesdw
        for i in range(epoch_count):
            self.loss.append( hist.history['loss'][i] )
        #Decay Epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
#Create the agent
nS = envCartPole.observation_space.shape[0] #This is only 4
nA = envCartPole.action_space.n #Actions
dqn = DeepQNetwork(nS, nA, learning_rate(), discount_rate(), 1, 0.001, 0.995 )

batch_size = batch_size()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#Training
import copy
rewards = [] #Store rewards for graphing
epsilons = [] # Store the Explore/Exploit
TEST_Episodes = 0
counter = 0
for e in range(EPISODES):
    state = envCartPole.reset()
    state = np.reshape(state, [1, nS]) # Resize to store in memory to pass to .predict
    tot_rewards = 0
    print("counter: ", counter)

#BEHOLD!!! This is where we increment the 
#do i call this surreal self, surreal self adversary or...??
    #add description
    if counter <3: 
        self_adversary = 0 # no adversary
    else: 
      #self_adversary = 0 # no adversary, aka equivalent to normal Q-learning
      self_adversary = 0.01#.0001
      self_adversary = self_adversary * counter #increment adversary
      if self_adversary >= .33 : #.47 : #1 is fully self-destructive. Philosophically fascinating but currently useless
        self_adversary = .33 
      ###


    for time in range(210): #200 is when you "solve" the game. This can continue forever as far as I know
        action = dqn.action(state)
        nstate, reward, done, _ = envCartPole.step(action)
        re_l_rwd = copy.deepcopy(reward)

        #the Surreal Adversary actually, "Attacks", here.
        #fully implicit, agent is unaware of this change to its reward
        #this is a case of Descartes Demon/Unreal Numbers. But I digress :)
        ###***
        reward = reward - ((reward * self_adversary)*2)
        ###***
        
        nstate = np.reshape(nstate, [1, nS])
        tot_rewards += re_l_rwd
        dqn.store(state, action, reward, nstate, done) # Resize to store in memory to pass to .predict
        state = nstate
        #done: CartPole fell. 
        #time == 209: CartPole stayed upright
        if done or time == 209:
            rewards.append(tot_rewards)
            epsilons.append(dqn.epsilon)
            print("episode: {}/{}, score: {}, e: {}, self_adversary: {}, reward: {}, re_l_rwd: {} time: {}"
                  .format(e, EPISODES, tot_rewards, dqn.epsilon, self_adversary, reward, re_l_rwd, time))
            break
        #Experience Replay
        if len(dqn.memory) > batch_size:
            dqn.experience_replay(batch_size)
    #If our current NN passes we are done
    #I am going to use the last 5 runs
    if len(rewards) > 5 and np.average(rewards[-5:]) > 195:
        #Set the rest of the EPISODES for testing
        TEST_Episodes = EPISODES - e
        TRAIN_END = e
        break
    counter = counter + 1 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


counter:  0
episode: 0/500, score: 29.0, e: 0.9801495006250001, self_adversary: 0, reward: 1.0, re_l_rwd: 1.0 time: 28
counter:  1
episode: 1/500, score: 15.0, e: 0.9137248860125932, self_adversary: 0, reward: 1.0, re_l_rwd: 1.0 time: 14
counter:  2
episode: 2/500, score: 23.0, e: 0.8183201210226743, self_adversary: 0, reward: 1.0, re_l_rwd: 1.0 time: 22
counter:  3
episode: 3/500, score: 29.0, e: 0.7111635524897149, self_adversary: 0.03, reward: 0.94, re_l_rwd: 1.0 time: 28
counter:  4
episode: 4/500, score: 13.0, e: 0.6696478204705644, self_adversary: 0.04, reward: 0.92, re_l_rwd: 1.0 time: 12
counter:  5
episode: 5/500, score: 18.0, e: 0.6149486215357263, self_adversary: 0.05, reward: 0.9, re_l_rwd: 1.0 time: 17
counter:  6
episode: 6/500, score: 13.0, e: 0.5790496471185967, self_adversary: 0.06, reward: 0.88, re_l_rwd: 1.0 time: 12
counter:  7
episode: 7/500, score: 9.0, e: 0.5562889678716474, self_adversary: 0.07, reward: 0.86, re_l_rwd: 1.0 time: 8
counter:  8
episode: 8/500, sco

In [ ]:
#Test the agent that was trained
#   In this section we ALWAYS use exploit don't train any more
for e_test in range(TEST_Episodes):
    state = envCartPole.reset()
    state = np.reshape(state, [1, nS])
    tot_rewards = 0
    for t_test in range(210):
        action = dqn.test_action(state)
        nstate, reward, done, _ = envCartPole.step(action)
        nstate = np.reshape( nstate, [1, nS])
        tot_rewards += reward
        #DON'T STORE ANYTHING DURING TESTING
        state = nstate
        #done: CartPole fell. 
        #t_test == 209: CartPole stayed upright
        if done or t_test == 209: 
            rewards.append(tot_rewards)
            epsilons.append(0) #We are doing full exploit
            print("episode: {}/{}, score: {}, e: {}"
                  .format(e_test, TEST_Episodes, tot_rewards, 0))
            break;

**Results**  
Here is a graph of the results. If everything was done correctly you should see the rewards over the red line.  

Black: This is the 100 episode rolling average  
Red: This is the "solved" line at 195  
Blue: This is the reward for each episode  
Green: This is the value of epsilon scaled by 200  
Yellow: This is where the tests started.

In [ ]:
rolling_average = np.convolve(rewards, np.ones(100)/100)

plt.plot(rewards, label='Episode Reward')
plt.plot(rolling_average, color='r', label='Rolling Avg')
plt.axhline(y=195, color='y', linestyle='-', label='Solved Line') #Solved Line
#Scale Epsilon (0.001 - 1.0) to match reward (0 - 200) range
eps_graph = [200*x for x in epsilons]
plt.plot(eps_graph, color='g',linestyle='-',label='Epsilon Decay')
#Plot the line where TESTING begins
#plt.axvline(x=TRAIN_END, color='o', linestyle='-')
plt.xlim( (0,EPISODES) )
plt.ylim( (0,220) )
plt.legend()
plt.title('Results for Single Surreal Agent')
plt.xlabel('Number of Episodes')
plt.ylabel('Score: Total Time per Episode')
plt.show()


envCartPole.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/Starcraft

In [ ]:
#pickle results so we can compare results of multiple runs
import pickle
pickle.dump( rewards, open("dqn_surreal_rewards_v5.txt", "wb" ) )
pickle.dump( rolling_average, open("dqn_surreal_rolling_average_v5.txt", "wb" ) )

In [ ]:
#load pickled results
loaded_rewards = pickle.load(open("dqn_surreal_rewards_v5.txt", "rb" ))
print(loaded_rewards)

loaded_moving_average = pickle.load(open("dqn_surreal_rolling_average_v5.txt", "rb" ))
print(loaded_moving_average)

**Changes**  
*hyper parameters*: You can alter alpha, gamma, batch size, and episode length to see what differences the algorithm returns.  
*Training End*: You can also change the line where I only check the last 5 runs before switching to testing mode (if len(rewards) > 5 and np.average(rewards[-5:]) > 195:) as that doesn't prove it was solved. The reason I did this was because I wanted to limit the amount of runs I made.  

**Conclusion**  
This is a Deep Q-Network implementation. There are some changes you can make here and there but it follows the paper. Hopefully, you were able to understand the code as well as make your own version to compare with this one.

**Reference**  
Mnih, V., Kavukcuoglu, K., Silver, D., Rusu, A. A., Veness, J., Bellemare, M. G., ... & Petersen, S. (2015). *Human-level control through deep reinforcement learning*. Nature, 518(7540), 529